In [1]:
import numpy as np
seed = 123
np.random.seed(seed)
import collections, copy, pickle
from importlib import reload
from dateutil.parser import parse
import scipy.linalg, scipy.stats
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 14
# rcParams['text.usetex'] = True
import seaborn as sns
from IPython.display import HTML

import scipy.stats
from sklearn import preprocessing

import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
import util.data
# import util.plot

In [14]:
data = pd.read_csv('data/training_set_VU_DM_clean.csv', sep=';', nrows=100000)
data_test = pd.read_csv('data/test_set_VU_DM.csv', sep=',', nrows=100000)

In [15]:
X = data

# drop values from X that are not present in test set
X = X.drop('score', 1)
X = X.drop('position', 1)
X = X.drop('click_bool', 1)
X = X.drop('booking_bool', 1)

y = data[['srch_id','score', 'prop_id']]

# create test and trainingset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=123)

# save the scores of the y train and test set in y_control, this is to calculate the ideal DCG for the test set
y_control = y_test
y_train = y_train.drop('srch_id', 1)
y_test = y_test.drop('srch_id', 1)
y_train = y_train.drop('prop_id', 1)
y_test = y_test.drop('prop_id', 1)

# data_dmatrix = xgb.DMatrix(data=X,label=y)

In [16]:
# use Gradient Boosting to make model of data to predict the 'score'
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1, 
                          max_depth = 5, alpha = 10, n_estimators = 10)

# fit training set
xg_reg.fit(X_train,y_train)

# fit model to test set
preds = xg_reg.predict(X_test)
X_test['score'] = preds

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [17]:
# sort test set predicted scores from high scores to low scores
X_test = X_test.sort_values(['srch_id', 'score'], ascending=[True, False])

# create new column to store position of prop_id
X_test['position'] = pd.Series()

# save position prop_id
prev_srch_id = -1
for i in X_test.index.tolist():
    row = X_test.loc[i]
    # compute position
    if prev_srch_id != row.srch_id:
        position = 1
        prev_srch_id = row.srch_id
    else:
        position += 1
        
    # save position value to X_test
    X_test.loc[i, 'position'] = int(position)

    # to calculate the DCG of the test set old scores have to be used instead of the predicted scores
    X_test.loc[i, 'score'] = y_control.loc[i, 'score']
    
# X_test

In [18]:
# determine ideal positions for test set with the 'real' scores, later the ideal DCG for the test set can be determined
y_control = y_control.sort_values(['srch_id', 'score'], ascending=[True, False])
y_control['position'] = pd.Series()
prev_srch_id = -1
for i in y_control.index.tolist():
    row = y_control.loc[i]
    # compute position
    if prev_srch_id != row.srch_id:
        position = 1
        prev_srch_id = row.srch_id
    else:
        position += 1

    # save value to X_test
    y_control.loc[i, 'position'] = int(position)


# tst = pd.concat([X_test[['srch_id', 'score','position', 'prop_id']], y_control], keys=['prop_id', 'X_test', 'y_control'], axis=1)

In [19]:
# calculate dcg of test set per srch_id
dcg_test = util.data.DCG_dict(X_test)

# calculate ideal dcg of test set per srch_id
dcg_control = util.data.DCG_dict(y_control)

# calculate means of both dcg dictionaries
print(np.mean(list(ndcg_test.values())))
print(np.mean(list(ndcg_control.values())))

# normalize
ndcg = np.mean(list(ndcg_test.values())) / np.mean(list(ndcg_control.values()))
ndcg

1.2435555133312008
1.6576291448209945


0.7502012842960064

In [20]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.917877
